In [133]:
import random
import torch
import torch.nn as nn
import torch.optim as optim

import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

# 读取数据集
u1_base = pd.read_csv('ml-100k/u1.base', sep='\t', names=['uid', 'iid', 'rate', 'timestamp'])
u1_test = pd.read_csv('ml-100k/u1.test', sep='\t', names=['uid', 'iid', 'rate', 'timestamp'])


#定义误差函数
def ERR(model, test):
    cnt = 0
    abs_err = 0
    squ_err = 0

    for index, row in test.iterrows():
        user_id = row['uid'] - 1
        item_id = row['iid'] - 1
        true_rating = row['rate']

        predicted_rating = model(user_id, item_id)
        if predicted_rating > 5:
            predicted_rating = 5
        if predicted_rating < 1:
            predicted_rating = 1

        absolute_error = abs(predicted_rating - true_rating)
        abs_err += absolute_error

        square_error = (predicted_rating - true_rating) ** 2
        squ_err += square_error

        cnt += 1

    mae = abs_err / cnt
    rmse = (squ_err / cnt) ** 0.5
    return mae, rmse


user_num = u1_base['uid'].max()
item_num = u1_base['iid'].max()

rating_matrix = np.zeros((user_num, item_num), float)
y_ui = np.zeros((user_num, item_num), int)

# 记录转换为矩阵
for index, row in u1_base.iterrows():
    user_id = row['uid']
    item_id = row['iid']
    rating = row['rate']
    rating_matrix[user_id - 1, item_id - 1] = rating
    y_ui[user_id - 1, item_id - 1] = 1

# Global average
GlobalAverage = rating_matrix.sum() / y_ui.sum()

#计算四个参数:user_means, item_means, user_bias, item_bias
rating_sum_row = [sum(row) for row in rating_matrix]
y_sum_row = [sum(row) for row in y_ui]

rating_sum_col = [sum(column) for column in zip(*rating_matrix)]
y_sum_col = [sum(column) for column in zip(*y_ui)]

user_means = []
for i in range(user_num):
    if y_sum_row[i] == 0:
        user_means.append(GlobalAverage)
    else:
        user_means.append(rating_sum_row[i] / y_sum_row[i])

item_means = []
for i in range(item_num):
    if y_sum_col[i] == 0:
        item_means.append(GlobalAverage)
    else:
        item_means.append(rating_sum_col[i] / y_sum_col[i])

user_bias = []
for i in range(user_num):
    if y_sum_row[i] == 0:
        user_bias.append(0)
    else:
        sum_bias = 0
        for j in range(item_num):
            sum_bias += y_ui[i][j] * (rating_matrix[i][j] - GlobalAverage)
        user_bias.append(sum_bias / y_sum_row[i])

item_bias = []
for i in range(item_num):
    if y_sum_col[i] == 0:
        item_bias.append(0)
    else:
        sum_bias = 0
        for j in range(user_num):
            sum_bias += y_ui[j][i] * (rating_matrix[j][i] - GlobalAverage)
        item_bias.append(sum_bias / y_sum_col[i])

In [134]:
# RSVD
class RSVD(nn.Module):
    def __init__(self, user_num, item_num, d, GlobalAverage, user_bias, item_bias):
        super(RSVD, self).__init__()
        self.U = nn.Parameter(torch.randn(user_num, d) * 0.01)
        self.V = nn.Parameter(torch.randn(item_num, d) * 0.01)
        self.user_bias = nn.Parameter(torch.tensor(user_bias, dtype=torch.float32))
        self.item_bias = nn.Parameter(torch.tensor(item_bias, dtype=torch.float32))
        self.GlobalAverage = GlobalAverage

    def forward(self, user_id, item_id):
        U_u = self.U[user_id]
        V_i = self.V[item_id]
        bu_u = self.user_bias[user_id]
        bi_i = self.item_bias[item_id]
        pred = torch.sum(U_u * V_i) + bu_u + bi_i + self.GlobalAverage
        return pred

In [135]:
d = 20
alpha_u = 0.01
alpha_v = 0.01
beta_u = 0.01
beta_v = 0.01
lr = 0.01
epochs = 60

#转换到tensor
train_data = torch.tensor(u1_base[['uid', 'iid']].values - 1, dtype=torch.long)
train_ratings = torch.tensor(u1_base['rate'].values, dtype=torch.float32)

# Dataset
train_dataset = TensorDataset(train_data, train_ratings)

# batch size = 2000
batch_size = 2000
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 定义模型、优化器、loss
model = RSVD(user_num, item_num, d, GlobalAverage, user_bias, item_bias)
optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=0.01)
criterion = nn.MSELoss()

In [136]:
# train
for epoch in range(epochs):
    model.train()
    total_loss = 0.0

    for batch_data, batch_ratings in train_loader:
        user_id, item_id = batch_data[:, 0], batch_data[:, 1]
        optimizer.zero_grad()

        predictions = model(user_id, item_id)

        loss = criterion(predictions, batch_ratings) #单个batch的loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {total_loss}')#一个epoch总loss
    if total_loss < 34.223:
        break
    lr *= 0.9

Epoch 1, Loss: 34.77484029531479
Epoch 2, Loss: 34.73276746273041
Epoch 3, Loss: 34.70992112159729
Epoch 4, Loss: 34.69286423921585
Epoch 5, Loss: 34.65987080335617
Epoch 6, Loss: 34.62838286161423
Epoch 7, Loss: 34.60522359609604
Epoch 8, Loss: 34.58187735080719
Epoch 9, Loss: 34.566838920116425
Epoch 10, Loss: 34.539741814136505
Epoch 11, Loss: 34.53562414646149
Epoch 12, Loss: 34.493729531764984
Epoch 13, Loss: 34.47113084793091
Epoch 14, Loss: 34.462037563323975
Epoch 15, Loss: 34.440704107284546
Epoch 16, Loss: 34.42167121171951
Epoch 17, Loss: 34.41071951389313
Epoch 18, Loss: 34.38404440879822
Epoch 19, Loss: 34.37974691390991
Epoch 20, Loss: 34.36994421482086
Epoch 21, Loss: 34.34861320257187
Epoch 22, Loss: 34.33832436800003
Epoch 23, Loss: 34.330923199653625
Epoch 24, Loss: 34.31173872947693
Epoch 25, Loss: 34.308274030685425
Epoch 26, Loss: 34.29112994670868
Epoch 27, Loss: 34.28686338663101
Epoch 28, Loss: 34.276792109012604
Epoch 29, Loss: 34.2761390209198
Epoch 30, Loss: 

In [137]:
#评价指标 ERR
mae, rmse = ERR(model,u1_test)
print(f'(RMSE:{rmse:.4f},MAE:{mae:.4f})')

(RMSE:0.9683,MAE:0.7670)
